## Import packages

In [ ]:
from omero.gateway import BlitzGateway
import ezomero

from dotenv import load_dotenv
import pandas as pd
# Import the function from your script
import json
import convert_excel_to_json
import importlib
importlib.reload(convert_excel_to_json)
from convert_excel_to_json import convert_excel_to_json



## Connect to OMERO

In [ ]:
load_dotenv(override=True)

conn = BlitzGateway(
    host=os.environ.get("HOST"),
    username=os.environ.get("USER_NAME"),
    passwd=os.environ.get("PASSWORD"),
    port=os.environ.get("PORT"),
    secure=True,
)
print(conn.connect())
conn.c.enableKeepAlive(60)

## choose a dataset


In [ ]:
datatype = "Dataset" # "plate", "dataset", "image"
data_id = 	502 #The ID of the Plate, Dataset, or Image you want to work with
nucl_channel = 0

#validate that data_id matches datatype
if datatype == "Plate":
    plate = conn.getObject("Plate", data_id)
    print('Plate Name: ', plate.getName())
elif datatype == "Dataset":
    dataset = conn.getObject("Dataset", data_id)
    print('Dataset Name: ', dataset.getName())
elif datatype == "Image":
    image = conn.getObject("Image", data_id)
    print('Image Name: ', image.getName())

In [ ]:
# Path to your Excel file
excel_file = " MIHCSME Template.xlsx"
# Convert Excel to JSON
result = convert_excel_to_json(excel_file)

# Display the result (optional)
print("Conversion completed successfully!")

# Save the result to a JSON file (optional)
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)


In [ ]:
import omero_metadata_uploader as omu
import ezomero
import json

# Load from Excel
excel_path = "MIHCSME Template_MH.xlsx"
json_data = omu.convert_excel_to_json(excel_path)

# Connect to OMERO

try:
    # Annotate at screen level
    screen_id = data_id
    omu.annotate_omero_screen(conn, screen_id, json_data, "metadata")
    
finally:
    conn.close()

In [ ]:
def camel_to_space(name):
    """
    Convert camelCase to space-separated words
    Keeps consecutive capital letters together (e.g., 'ID', 'RNA')
    """
    result = []
    # Track if we're in a sequence of capitals
    in_capitals = False
    
    for i, char in enumerate(name):
        # Check if this is a capital letter
        if char.isupper():
            # Check if previous character exists and was not a capital
            if i > 0 and not in_capitals:
                result.append(' ')
            in_capitals = True
        else:
            in_capitals = False
            
        result.append(char)
    
    return ''.join(result).strip()

def read_metadata_template(excel_path):
    """
    Read Excel metadata template and convert to a dictionary format
    for OMERO annotations, specifically from AssayConditions tab.
    Converts column headers from camelCase to space-separated words.
    """
    # Read the full sheet without assuming header position
    df = pd.read_excel(excel_path, sheet_name='AssayConditions', header=None)
    
    # Find the row containing the actual column headers
    header_row = df[df[0] == 'Plate'].index[0]
    
    # Read the Excel again, now with correct header position
    df = pd.read_excel(excel_path, 
                      sheet_name='AssayConditions', 
                      header=header_row)
    
    # Rename columns to convert from camelCase
    df.columns = [camel_to_space(col) if col != 'Well' else col for col in df.columns]
    
    # Convert DataFrame to dictionary where keys are Well IDs
    metadata_dict = df.set_index('Well').to_dict('index')
    return metadata_dict

def add_metadata_to_object(conn, object_type, object_id, metadata, namespace):
    """
    Add or update metadata as map annotation to an OMERO object using ezomero.
    Preserves existing keys that are not in the new metadata.
    
    Args:
        conn: OMERO connection
        object_type: String indicating OMERO object type (e.g., "Image", "Dataset")
        object_id: ID of the OMERO object
        metadata: Dictionary of metadata key-value pairs
        namespace: Namespace for the annotation
    """
    # Get existing map annotation IDs
    existing_ann_ids = ezomero.get_map_annotation_ids(
        conn,
        object_type,
        object_id,
        ns=namespace
    )
    
    # Initialize merged metadata dictionary
    merged_metadata = {}
    
    if existing_ann_ids:
        # Get the first annotation values
        obj = conn.getObject(object_type, object_id)
        links_to_delete = []
        
        for ann in obj.listAnnotations(ns=namespace):
            if ann.id == existing_ann_ids[0]:  # Use first annotation if multiple exist
                merged_metadata = dict(ann.getValue())
                links_to_delete.append(ann.link.id)
                break
        
        # Unlink old annotations
        if links_to_delete:
            print(f"Unlinking {len(links_to_delete)} annotations")
            conn.deleteObjects("ImageAnnotationLink", links_to_delete, wait=True)
    
    # Filter out any NaN values from the new metadata
    clean_metadata = {k: str(v) for k, v in metadata.items() if pd.notna(v)}
    
    # Update merged metadata with new values
    merged_metadata.update(clean_metadata)
    
    # Create new annotation with merged metadata
    map_ann_id = ezomero.post_map_annotation(
        conn,
        object_type,
        object_id,
        merged_metadata,
        namespace
    )
    return map_ann_id

def remove_metadata_from_images(conn, dataset_id, namespace=None, delete_annotations=True):
    """
    Remove map annotations from all images in a dataset
    
    Args:
        conn: OMERO connection
        dataset_id: ID of the dataset containing images
        namespace: Optional - specific namespace to remove. If None, removes all map annotations
        delete_annotations: If True, deletes the annotations. If False, just unlinks them
    """
    # Get all images from dataset
    dataset = conn.getObject("Dataset", dataset_id)
    images = list(dataset.listChildren())
    print(f"Found {len(images)} images in dataset")
    
    for image in images:
        print(f"Processing image {image.getName()}")
        
        # Get annotations
        to_delete = []
        for ann in image.listAnnotations(ns=namespace):
            if delete_annotations:
                to_delete.append(ann.id)
            else:
                to_delete.append(ann.link.id)
        
        if to_delete:
            if delete_annotations:
                # Delete the annotations completely
                print(f"Deleting {len(to_delete)} annotations")
                conn.deleteObjects("Annotation", to_delete, wait=True)
            else:
                # Just unlink the annotations
                print(f"Unlinking {len(to_delete)} annotations")
                conn.deleteObjects("ImageAnnotationLink", to_delete, wait=True)

def remove_screen_metadata(conn, screen_id, namespace=None, delete_annotations=True):
    """
    Remove map annotations from a screen, its plates, and wells
    
    Args:
        conn: OMERO connection
        screen_id: ID of the screen
        namespace: Optional - specific namespace to remove. If None, removes all map annotations
        delete_annotations: If True, deletes the annotations. If False, just unlinks them
    """
    # Get screen
    screen = conn.getObject("Screen", screen_id)
    if not screen:
        print(f"Screen {screen_id} not found")
        return
    
    # Remove annotations from screen
    to_delete = []
    for ann in screen.listAnnotations(ns=namespace):
        if delete_annotations:
            to_delete.append(ann.id)
        else:
            to_delete.append(ann.link.id)
    
    if to_delete:
        if delete_annotations:
            print(f"Deleting {len(to_delete)} annotations from Screen {screen_id}")
            conn.deleteObjects("Annotation", to_delete, wait=True)
        else:
            print(f"Unlinking {len(to_delete)} annotations from Screen {screen_id}")
            conn.deleteObjects("ScreenAnnotationLink", to_delete, wait=True)
    
    # Get plates in screen
    plates = list(screen.listChildren())
    print(f"Found {len(plates)} plates in screen")
    
    # Process each plate
    for plate in plates:
        print(f"Processing plate {plate.getName()}")
        
        # Remove annotations from plate
        plate_to_delete = []
        for ann in plate.listAnnotations(ns=namespace):
            if delete_annotations:
                plate_to_delete.append(ann.id)
            else:
                plate_to_delete.append(ann.link.id)
        
        if plate_to_delete:
            if delete_annotations:
                print(f"Deleting {len(plate_to_delete)} annotations from plate")
                conn.deleteObjects("Annotation", plate_to_delete, wait=True)
            else:
                print(f"Unlinking {len(plate_to_delete)} annotations from plate")
                conn.deleteObjects("PlateAnnotationLink", plate_to_delete, wait=True)
        
        # Get wells in plate
        wells = []
        for well in plate.listChildren():
            wells.append(well)
        
        print(f"Found {len(wells)} wells in plate")
        
        # Process each well
        for well in wells:
            # Remove annotations from well
            well_to_delete = []
            for ann in well.listAnnotations(ns=namespace):
                if delete_annotations:
                    well_to_delete.append(ann.id)
                else:
                    well_to_delete.append(ann.link.id)
            
            if well_to_delete:
                if delete_annotations:
                    print(f"Deleting {len(well_to_delete)} annotations from well {well.id}")
                    conn.deleteObjects("Annotation", well_to_delete, wait=True)
                else:
                    print(f"Unlinking {len(well_to_delete)} annotations from well {well.id}")
                    conn.deleteObjects("WellAnnotationLink", well_to_delete, wait=True)

In [ ]:
# Read metadata template
excel_path = "MIHCSME Template.xlsx"
metadata_dict = read_metadata_template(excel_path)

# Define namespace for annotations
namespace = "openmicroscopy.org/omero/experimental/metadata"

# Get all images from dataset
dataset = conn.getObject("Dataset", data_id)
images = list(dataset.listChildren())  # Convert to list for length check and reuse
num_images = len(images)
num_metadata_rows = len(metadata_dict)

print(f"Found {num_images} images in dataset")
print(f"Found {num_metadata_rows} rows in metadata")

if num_images != num_metadata_rows:
    print("Warning: Number of images doesn't match number of metadata rows!")
    print("Please check your data before proceeding.")
else:
    # Convert metadata_dict values to list to access by index
    metadata_list = list(metadata_dict.values())
    
    # Loop through images and metadata together
    for i, image in enumerate(images):
        # Get metadata for this position
        metadata = metadata_list[i]
        
        # Add metadata to image
        map_ann_id = add_metadata_to_object(
            conn,
            "Image",
            image.getId(),
            metadata,
            namespace
        )
        print(f"Added metadata to image {image.getName()} with annotation ID {map_ann_id}")

## Remove all metadata annotations

In [ ]:

# Usage example:
# To delete annotations:
# remove_metadata_from_images(
#     conn, 
#     data_id,
#     namespace="openmicroscopy.org/omero/experimental/metadata",
#     delete_annotations=True
# )
remove_screen_metadata(conn, data_id, namespace=None, delete_annotations=True)
# To just unlink annotations:
# remove_metadata_from_images(
#     conn, 
#     dataset_id,
#     namespace="openmicroscopy.org/omero/experimental/metadata",
#     delete_annotations=False
# )